In [17]:
import pandas as pd
from difflib import SequenceMatcher

In [18]:
#Importation des datasets
df  = pd.read_csv(r"data\dataset_final_v1.csv").drop(columns="Unnamed: 0") 
actor = pd.read_csv("data//actor_scrap.csv").drop(columns="Unnamed: 0")
director = pd.read_csv(r"data\director_scrap.csv").drop(columns="Unnamed: 0")
compagnies = pd.read_csv(r'data\compagny_scrap.csv').drop(columns="Unnamed: 0")

df.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,...,Western,budget,casting,compagnies_production,date,duree,realisateur,recette,suite,titre
0,0,0,0,1,0,0,0,0,0,0,...,0,40000000,"['Jim Carrey', 'Jeff Daniels', 'Laurie Holden'...","['New Line Cinema', 'Universal Pictures', 'Con...",2014-11-12,110,"['Bobby Farrelly', 'Peter Farrelly']",169837010,3,Dumb and Dumber To
1,0,0,0,1,0,0,1,0,0,0,...,0,2000000,"['Nana Kiknadze', 'Artur Smolyaninov', 'Yevgen...","['Central Partnership', 'Studio SLON']",2004-11-11,100,['Anna Melikyan'],240000,1,Марс
2,1,1,0,0,0,0,0,0,0,0,...,0,170000000,"['Chris Evans', 'Scarlett Johansson', 'Sebasti...",['Marvel Studios'],2014-03-20,136,"['Anthony Russo', 'Joe Russo']",714766572,2,Captain America: The Winter Soldier
3,0,0,0,1,0,0,0,1,1,0,...,0,4000000,"['Mikhail Galustyan', 'Igor Vernik', 'Mariya S...",['Enjoy Movies'],2012-03-15,80,['Sarik Andreasyan'],9938268,1,Тот ещё Карлосон!
4,1,1,0,0,0,0,0,0,0,0,...,0,68000000,"['Val Kilmer', 'Elisabeth Shue', 'Rade Šerbedž...","['Paramount', 'Mace Neufeld Productions', 'Rys...",1997-04-03,116,['Phillip Noyce'],118063304,1,The Saint


In [19]:
list_keys = ['realisateur', 'casting', 'compagnies_production']

def str_to_list(df, key):
    df_new = df

    for k in key: #on parcourt les clés à modifier spécifiées en argument de la fonction

        for ind in range(df_new.shape[0]): #on parcourt toutes les lignes du Dataframe

            string = df_new[k].iloc[ind] #on récupère la valeur à modifier pour chaque ligne

            if type(string) == str : # on vérifie si c'est bien une string
                
                #création de la nouvelle variable de type liste qui remplacera l'ancienne

                liste_finale = []

                #remplissage de la variable de type liste en faisant des opérations sur la string

                list_string = string[1:-1].split(',')
                for i,j in enumerate(list_string):
                    if i == 0:
                        mot = j[1:-1]
                        if mot not in liste_finale:
                            liste_finale.append(mot)
                    else:
                        mot = j[2:-1]
                        if mot not in liste_finale:
                            liste_finale.append(mot)
                
                #affectation de la nouvelle valeur de type liste à l'élément modifié du Dataframe

                df_new.at[ind,k] = liste_finale

    return df_new

df = str_to_list(df, list_keys)

In [20]:
compagnies.head()

,Compagny_Name,Worldwide_box_office
0,Warner Bros.,48055413164
1,Universal Pictures,47901646442
2,Columbia Pictures,45406879151
3,Walt Disney Pictures,39988482983
4,Paramount Pictures,35708606956


In [21]:
#Contruction des notes des acteurs/réalisateur
#Acteur
actor["rating"] = 100*actor["Worldwide_box_office"]/actor['Worldwide_box_office'].max()
actor["rating"] = actor["rating"].astype(int)

#Réalisateur
director["rating"] = 100*director["Worldwide_box_office"]/director['Worldwide_box_office'].max()
director["rating"] = director["rating"].astype(int)

#Compagnies
compagnies["rating"] = 100*compagnies["Worldwide_box_office"]/compagnies['Worldwide_box_office'].max()
compagnies["rating"] = compagnies["rating"].astype(int)


In [22]:
actor = actor[actor["rating"] > 0]
director = director[director["rating"] > 0]
compagnies = compagnies[compagnies["rating"] > 0]
compagnies.head()

,Compagny_Name,Worldwide_box_office,rating
0,Warner Bros.,48055413164,100
1,Universal Pictures,47901646442,99
2,Columbia Pictures,45406879151,94
3,Walt Disney Pictures,39988482983,83
4,Paramount Pictures,35708606956,74


In [31]:
"""actor.to_csv("data/actor_ratings.csv")
director.to_csv("data/director_ratings.csv")
compagnies.to_csv("data/compagnies_ratings.csv")"""

'actor.to_csv("data/actor_ratings.csv")\ndirector.to_csv("data/director_ratings.csv")\ncompagnies.to_csv("data/compagnies_ratings.csv")'

In [32]:
#Création des dictionnaires
actor_dict = {}
for i, row in actor.iterrows():
    actor_dict[row.Actor_Name] = row.rating

director_dict = {}
for i, row in director.iterrows():
   director_dict[row.Director_Name] = row.rating

compagnies_dict = {}
for i, row in compagnies.iterrows():
   compagnies_dict[row.Compagny_Name] = row.rating



In [33]:
def comparaison_string(str1, str2): #Renvoie la ressemblance en deux strings
    S = SequenceMatcher(None,  str1, str2)
    if S.ratio() > 0.8 :
        return True
    else :
        return False

def comparaison_dict(string, dict): #Renvoie le rating associé à un acteur
    for name, rating in dict.items():
        if comparaison_string(string, name):
            return rating
    return 0

In [34]:
#On remplace maintenant les données par leur score
for i, row in df.iterrows():
    sum_actor = 0
    sum_director = 0
    sum_compagnies = 0
    for actor in row.casting:
        sum_actor += comparaison_dict(actor, actor_dict)
    for director in row.realisateur:
        sum_director += comparaison_dict(director, director_dict)
    for compagny in row.compagnies_production:
        sum_compagnies += comparaison_dict(compagny, compagnies_dict)
    
    df.loc[i, 'casting_score'] = int(sum_actor)
    df.loc[i, 'realisateur_score'] = int(sum_director)
    df.loc[i, 'compagny_score'] = int(sum_compagnies)
df.to_csv("arrange.csv")

In [21]:
df2  = pd.read_csv(r"C:\Users\kerac\OneDrive\IMT\A2\Ue-Journey\Projet\Code\MovieHype\arrange.csv").drop(columns="Unnamed: 0") 
df2.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,...,compagnies_production,date,duree,realisateur,recette,suite,titre,casting_score,realisateur_score,compagny_score
0,0,0,0,1,0,0,0,0,0,0,...,"['New Line Cinema', 'Universal Pictures', 'Con...",2014-11-12,110,"['Bobby Farrelly', 'Peter Farrelly']",169837010,3,Dumb and Dumber To,79.0,27.0,135.0
1,1,1,0,0,0,0,0,0,0,0,...,['Marvel Studios'],2014-03-20,136,"['Anthony Russo', 'Joe Russo']",714766572,2,Captain America: The Winter Soldier,459.0,128.0,81.0
2,0,0,0,1,0,0,0,1,1,0,...,['Enjoy Movies'],2012-03-15,80,['Sarik Andreasyan'],9938268,1,Тот ещё Карлосон!,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,"['Lions Gate Films', 'WWE Studios']",2006-05-19,84,['Gregory Dark'],18707966,1,See No Evil,5.0,0.0,0.0
4,0,1,1,0,0,0,0,1,0,0,...,['Walt Disney Feature Animation'],2003-10-23,85,"['Aaron Blaise', 'Robert Walker']",250397798,1,Brother Bear,24.0,0.0,0.0
